axis_info:轴信息但不知道是什么轴

adc_info:模数转换的一些参数

train_labels:应该就是需要预测的光谱强度

wavelength:每一个预测的列对应的光的波长

每一个子文件提供了传感器接受到的像素点的强度，并拉长为一行，此外另一个维度应该是时间。子文件中的calibration提供了传感器的一些校准文件：

dark表示热噪声，

dead表示无效的像素，

flat表示提供一个均匀的光照板，测试传感器得到的光照强度以此校准，

linear表示传感器产生的非线性？由于使用导致传感器的输出值可能不是随着光照线性增加的，所以利用一个n次多项式进行修正？

read项表示传感器读出过程中的电子噪声


这样的话是不是意味着我们需要使用signal信息预测label值，然后利用用于校准的数据预测可能的标准差？

In [13]:
import numpy as np
import polars as pl
import pandas as pd

import os
import gc

In [2]:
axis_info = pl.read_parquet('/kaggle/input/ariel-data-challenge-2024/axis_info.parquet')

In [7]:
pd_axis_info = axis_info.to_pandas()

In [10]:
pd_axis_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135000 entries, 0 to 134999
Data columns (total 4 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   AIRS-CH0-axis0-h           11250 non-null   float64
 1   AIRS-CH0-axis2-um          356 non-null     float64
 2   AIRS-CH0-integration_time  11250 non-null   float64
 3   FGS1-axis0-h               135000 non-null  float64
dtypes: float64(4)
memory usage: 4.1 MB


In [15]:
pd_axis_info.dropna()

,AIRS-CH0-axis0-h,AIRS-CH0-axis2-um,AIRS-CH0-integration_time,FGS1-axis0-h
0,0.000028,4.078463,0.1,0.000028
1,0.000056,4.074023,4.5,0.000056
2,0.001361,4.069568,0.1,0.000139
3,0.001389,4.065100,4.5,0.000167
4,0.002694,4.060618,0.1,0.000250
...,...,...,...,...
351,0.233389,1.670511,4.5,0.019500
352,0.234694,1.661217,0.1,0.019583
353,0.234722,1.651909,4.5,0.019611
354,0.236028,1.642588,0.1,0.019694


In [18]:
pd.read_csv('/kaggle/input/ariel-data-challenge-2024/train_labels.csv')

,planet_id,wl_1,wl_2,wl_3,wl_4,wl_5,wl_6,wl_7,wl_8,wl_9,...,wl_274,wl_275,wl_276,wl_277,wl_278,wl_279,wl_280,wl_281,wl_282,wl_283
0,785834,0.001086,0.001137,0.001131,0.001124,0.001138,0.001131,0.001123,0.001127,0.001120,...,0.001075,0.001076,0.001076,0.001076,0.001074,0.001073,0.001072,0.001073,0.001073,0.001072
1,14485303,0.001835,0.001835,0.001834,0.001833,0.001833,0.001833,0.001833,0.001834,0.001834,...,0.001893,0.001892,0.001892,0.001891,0.001891,0.001891,0.001890,0.001890,0.001889,0.001888
2,17002355,0.002792,0.002814,0.002808,0.002804,0.002809,0.002805,0.002802,0.002805,0.002801,...,0.002784,0.002783,0.002783,0.002783,0.002783,0.002784,0.002784,0.002785,0.002785,0.002784
3,24135240,0.001294,0.001308,0.001308,0.001306,0.001306,0.001303,0.001306,0.001314,0.001314,...,0.001405,0.001404,0.001403,0.001402,0.001401,0.001400,0.001399,0.001397,0.001395,0.001393
4,25070640,0.001987,0.001987,0.001987,0.001987,0.001987,0.001987,0.001987,0.001987,0.001987,...,0.001988,0.001988,0.001988,0.001988,0.001988,0.001988,0.001988,0.001988,0.001988,0.001988
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668,4249337798,0.000790,0.000828,0.000825,0.000822,0.000830,0.000826,0.000822,0.000823,0.000820,...,0.000798,0.000799,0.000799,0.000799,0.000798,0.000796,0.000796,0.000796,0.000796,0.000796
669,4257395405,0.004063,0.004094,0.004090,0.004094,0.004083,0.004069,0.004076,0.004097,0.004100,...,0.004042,0.004042,0.004041,0.004041,0.004041,0.004041,0.004041,0.004040,0.004040,0.004039
670,4266129805,0.006566,0.006569,0.006568,0.006568,0.006567,0.006567,0.006568,0.006570,0.006569,...,0.006566,0.006566,0.006566,0.006566,0.006566,0.006566,0.006566,0.006566,0.006566,0.006566
671,4273166473,0.005647,0.005700,0.005687,0.005676,0.005687,0.005676,0.005667,0.005673,0.005663,...,0.005603,0.005603,0.005603,0.005602,0.005602,0.005604,0.005606,0.005607,0.005607,0.005605


In [19]:
pd.read_csv('/kaggle/input/ariel-data-challenge-2024/wavelengths.csv')

,wl_1,wl_2,wl_3,wl_4,wl_5,wl_6,wl_7,wl_8,wl_9,wl_10,...,wl_274,wl_275,wl_276,wl_277,wl_278,wl_279,wl_280,wl_281,wl_282,wl_283
0,0.705,1.951761,1.960612,1.96945,1.978273,1.987083,1.995879,2.004661,2.01343,2.022184,...,3.84972,3.85481,3.859887,3.86495,3.869999,3.875034,3.880055,3.885063,3.890056,3.895036
